In [ ]:
!pip3 install pandas openpyxl bertopic setuptools 'numpy<2' nltk spacy textstat textblob torch matplotlib ace_tools 
# add this to terminal every time - source .venv/bin/activate
#!pip3 install --upgrade pip

Combine all the data [This step may not be required after Berke sends the overall merged file] 

In [1]:
import os
import pandas as pd
import re
from pathlib import Path

# 1. Set folder path and list files
folder_path = "/Users/vinitavader/Desktop/WJC/data"
file_paths = list(Path(folder_path).glob("*.xlsx"))

# 2. Set of countries that have translated responses
translated_countries = {"Brazil", "China", "Colombia", "Kazakhastan", "Poland", "Turkey"}

# 3. Process each file
def process_file(file_path):
    country = file_path.name.split('_')[0]  # Preserve original casing

    try:
        df = pd.read_excel(file_path)

        # 4. Define column pattern depending on translation status
        if country in translated_countries:
            # Look for _EN columns only
            lb_cols = [col for col in df.columns if re.match(r"LB_Open_Q[1-7]_EN$", col, flags=re.IGNORECASE)]
            smb_cols = [col for col in df.columns if re.match(r"SMB_Open_Q[1-7]_EN$", col, flags=re.IGNORECASE)]
        else:
            # Use standard non-EN columns
            lb_cols = [col for col in df.columns if re.match(r"LB_Open_Q[1-7]$", col, flags=re.IGNORECASE)]
            smb_cols = [col for col in df.columns if re.match(r"SMB_Open_Q[1-7]$", col, flags=re.IGNORECASE)]

        selected_cols = lb_cols + smb_cols
        if not selected_cols:
            print(f"⚠️ Skipping {file_path.name}: no relevant columns found")
            return None

        df = df[selected_cols].copy()

        # 5. Remove "_EN" suffix if present
        df.columns = [re.sub(r"_EN$", "", col, flags=re.IGNORECASE) for col in df.columns]

        # 6. Add pID and country
        df.insert(0, "pID", [f"{country}_p{i+1}" for i in range(len(df))])
        df.insert(1, "country", country)

        return df

    except Exception as e:
        print(f"❌ Skipping file {file_path.name}: {e}")
        return None

# 7. Collect all valid dataframes
dfs = []
for file_path in file_paths:
    df = process_file(file_path)
    if df is not None:
        if df.columns.duplicated().any():
            print(f"⚠️ Duplicate columns in {file_path.name}:")
            print(df.columns[df.columns.duplicated()].tolist())
        else:
            dfs.append(df)

# 8. Combine into one dataframe
combined_df = pd.concat(dfs, ignore_index=True)

# 8. Replace empty strings with NaN
lb_cols = [f"LB_Open_Q{i}" for i in range(1, 8)]
smb_cols = [f"SMB_Open_Q{i}" for i in range(1, 8)]
combined_df[lb_cols + smb_cols] = combined_df[lb_cols + smb_cols].replace('', pd.NA)

# 9. Create Open_Q1 to Q7 using LB fallback to SMB
for i in range(1, 8):
    combined_df[f"Open_Q{i}"] = (
        combined_df.get(f"LB_Open_Q{i}").fillna(combined_df.get(f"SMB_Open_Q{i}")).astype(str)
    )

# 10. Unite Q1–Q7 and Q1–Q7_no6
open_q1_7_no6 = [f"Open_Q{i}" for i in range(1, 8) if i != 6]
open_q1_7 = [f"Open_Q{i}" for i in range(1, 8)]

combined_df["Open_Q1_Q7_no6"] = combined_df[open_q1_7_no6].apply(lambda row: ' '.join(row.dropna()), axis=1)
combined_df["Open_Q1_Q7"] = combined_df[open_q1_7].apply(lambda row: ' '.join(row.dropna()), axis=1)

# 11. Final dataframe selection
text_combined_df = combined_df[['pID', 'country'] + lb_cols + smb_cols + ['Open_Q1_Q7_no6', 'Open_Q1_Q7']]

text_combined_df

,pID,country,LB_Open_Q1,LB_Open_Q2,LB_Open_Q3,LB_Open_Q4,LB_Open_Q5,LB_Open_Q6,LB_Open_Q7,SMB_Open_Q1,SMB_Open_Q2,SMB_Open_Q3,SMB_Open_Q4,SMB_Open_Q5,SMB_Open_Q6,SMB_Open_Q7,Open_Q1_Q7_no6,Open_Q1_Q7
0,Turkey_p1,Turkey,I had a South American boyfriend in my early 2...,Disappointment and fear of telling my family a...,I felt stuck. First the shock of being cheated...,"I felt a huge disappointment, dilemma, helples...",At first I tried to accept the situation. Okay...,It both forced and transformed me. I questione...,no there's not,NaN,NaN,NaN,NaN,NaN,NaN,NaN,I had a South American boyfriend in my early 2...,I had a South American boyfriend in my early 2...
1,Turkey_p2,Turkey,There was a time when my brother had to be hos...,"Uncertainty, fear, anxiety, resistance from my...",Our whole life is turned upside down,Sadness,"First of all, when we realized my brother's co...","At that time, it was like not knowing what to ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,There was a time when my brother had to be hos...,There was a time when my brother had to be hos...
2,Turkey_p3,Turkey,Resigning from the name because I had a baby w...,Everyone intervened. They gave advice and trie...,I wished I wouldn't regret it. I was afraid of...,I felt indecisive and helpless.,I didn't listen to anyone and resigned for my ...,It affected me financially. I felt very empty.,Thank God I didn't regret it. I started workin...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Resigning from the name because I had a baby w...,Resigning from the name because I had a baby w...
3,Turkey_p4,Turkey,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"I was stuck between my ex-wife and my family, ...",Family is more important than anything else,I thought I was not loved,I was saddened by the pessimism,I love my brothers so much I would die for them,Sad and pessimistic,NaN,"I was stuck between my ex-wife and my family, ...","I was stuck between my ex-wife and my family, ..."
4,Turkey_p5,Turkey,I had a hard time deciding where to be assigne...,I chose the one with a good salary because I w...,"Without thinking, I directly chose the good pr...",I felt complicated because life was going to b...,I went for the one that was difficult but paid...,My life from now on will be more difficult. I ...,Maybe in the future I will want to go to the o...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,I had a hard time deciding where to be assigne...,I had a hard time deciding where to be assigne...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1457,UK_p147,UK,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Whether to invest in shares or not,There was very little room for error financial...,It could be the way to resolve my financial si...,Happy. It seemed like a possible solution!,Looked into several options for investment. To...,"It has been anxious, not very nice.",I hope it pays off!,Whether to invest in shares or not There was v...,Whether to invest in shares or not There was v...
1458,UK_p148,UK,The most difficult decision I ever made was wh...,Trying to weigh up whether what we would be gi...,I was originally thinking that I did not reall...,I was feeling anxious about it as my now wife ...,Ultimately I made the decision that despite my...,It changed my life forever but ultimately it t...,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN,The most difficult decision I ever made was wh...,The most difficult decision I ever made was wh...
1459,UK_p149,UK,I was offered voluntary redundancy from the fi...,I had to decide if I wanted to leave a secure ...,I was thinking about my future and what I coul...,I felt anxious as I do not like change and the...,I discussed it with family and friends. I we...,I lost a lot of my confidence in my ability to...,no,NaN,NaN,NaN,NaN,NaN,NaN,NaN,I was offered voluntary redundancy from the fi...,I was offered voluntary redundancy from the fi...
1460,UK_p150,UK,NaN,NaN,NaN,NaN,NaN,NaN,NaN,We booked a hotel. When we got to the hotel it...,The main problem was finding replacement accom...,I was thinking that the hotel was terrib

# Classify Text for the life domains
What are the domains that people talk about when it comes to difficult like decisions?
For example - Family, Career etc

In [ ]:
import pandas as pd
from transformers import pipeline

# 1. Define decision categories
DEC_CATS = {
    'career':      ['job','work','career','profession','employment','business'],
    'relationships':['relationship','marriage','divorce','dating','partner','love'],
    'family':      ['family','children','parents','siblings','relatives'],
    'health':      ['health','medical','illness','doctor','treatment','surgery'],
    'financial':   ['money','financial','investment','debt','loan','purchase'],
    'education':   ['school','university','education','degree','study'],
    'location':    ['move','relocate','city','country','home','house'],
    'ethical':     ['right','wrong','moral','ethical','values','principle']
}

# 2. Count keywords in each category
def decision_cats(text):
    text_l = text.lower()
    return {f"dec_cat_{cat}": sum(text_l.count(kw) for kw in kws)
            for cat, kws in DEC_CATS.items()}

# 3. Apply keyword counts
dec_df = text_combined_df["Open_Q1_Q7"].apply(decision_cats).apply(pd.Series)
text_combined_df = pd.concat([text_combined_df, dec_df], axis=1)

# 4. Zero-shot classification
zsc = pipeline(
    "zero-shot-classification",
    model="facebook/bart-large-mnli",
    device=0  # use -1 for CPU # defaulting to CPU as I don't have GPU
)
candidate_labels = list(DEC_CATS.keys())

def classify_zero_shot(text):
    out = zsc(text, candidate_labels=candidate_labels, multi_label=False)
    return out["labels"][0]

texts = text_combined_df["Open_Q1_Q7"].tolist()
results = zsc(texts, candidate_labels=candidate_labels, multi_label=True)

text_combined_df["llm_category"] = [res["labels"][0] for res in results]


# 5. generate *_count and *_llm vars
count_vars = {f"{cat}_count": text_combined_df[f"dec_cat_{cat}"] for cat in DEC_CATS}
llm_vars   = {f"{cat}_llm": (text_combined_df["llm_category"] == cat).astype(int) for cat in DEC_CATS}

# 6. Example usage
print(count_vars["career_count"].head())
print(llm_vars["career_llm"].head())


In [ ]:
import pandas as pd
from transformers import pipeline

# Subset for testing
subset_df = text_combined_dfthe 

# Decision categories
DEC_CATS = {
    'career':      ['job','work','career','profession','employment','business'],
    'relationships':['relationship','marriage','divorce','dating','partner','love'],
    'family':      ['family','children','parents','siblings','relatives'],
    'health':      ['health','medical','illness','doctor','treatment','surgery'],
    'financial':   ['money','financial','investment','debt','loan','purchase'],
    'education':   ['school','university','education','degree','study'],
    'location':    ['move','relocate','city','country','home','house'],
    'ethical':     ['right','wrong','moral','ethical','values','principle']
}
candidate_labels = list(DEC_CATS.keys())

# 1. Keyword-based category counts
def decision_cats(text):
    text_l = text.lower()
    return {f"count_cat_{cat}": sum(text_l.count(kw) for kw in kws) for cat, kws in DEC_CATS.items()}
count_df = subset_df["Open_Q1_Q7"].apply(decision_cats).apply(pd.Series)

# Add most-frequent category based on count
subset_df["count_category"] = count_df.idxmax(axis=1).str.replace("count_cat_", "")

# 2. Zero-shot classification
zsc = pipeline(
    "zero-shot-classification",
    model="facebook/bart-large-mnli",
    device=-1
)

texts = subset_df["Open_Q1_Q7"].tolist()
results = zsc(texts, candidate_labels=candidate_labels, multi_label=True)

# Top predicted category
subset_df["llm_category"] = [r["labels"][0] for r in results]

# Score columns
score_df = pd.DataFrame([
    {f"llm_cat_{label}": score for label, score in zip(r["labels"], r["scores"])}
    for r in results
])

# Combine everything
subset_df = pd.concat([subset_df, count_df, score_df], axis=1)

# Export
subset_df.to_excel("llm_vs_count_decision_categories.xlsx", index=False)

# Preview
print(subset_df[["Open_Q1_Q7", "count_category", "llm_category"] + list(score_df.columns)].head())


In [6]:
# --- Task 1: Count-Based Category Detection ---
import pandas as pd

# Your data
text_df = text_combined_df.copy()

# Define keyword-based decision categories
DEC_CATS = {
    'career':      ['job','work','career','profession','employment','business'],
    'relationships':['relationship','marriage','divorce','dating','partner','love'],
    'family':      ['family','children','parents','siblings','relatives'],
    'health':      ['health','medical','illness','doctor','treatment','surgery'],
    'financial':   ['money','financial','investment','debt','loan','purchase'],
    'education':   ['school','university','education','degree','study'],
    'location':    ['move','relocate','city','country','home','house'],
    'ethical':     ['right','wrong','moral','ethical','values','principle']
}

# Keyword count function
def decision_cats(text):
    text_l = text.lower()
    return {f"count_cat_{cat}": sum(text_l.count(kw) for kw in kws) for cat, kws in DEC_CATS.items()}

# Apply count-based classifier
count_df = text_df["Open_Q1_Q7"].apply(decision_cats).apply(pd.Series)

# Add top count-based category
text_df["count_category"] = count_df.idxmax(axis=1).str.replace("count_cat_", "")

# Combine results
text_df = pd.concat([text_df, count_df], axis=1)

# Save (optional)
text_df.to_excel("count_based_classification.xlsx", index=False)

# Preview
print(text_df[["Open_Q1_Q7", "count_category"] + list(count_df.columns)].head())


                                          Open_Q1_Q7 count_category  \
0  I had a South American boyfriend in my early 2...  relationships   
1  There was a time when my brother had to be hos...        ethical   
2  Resigning from the name because I had a baby w...         career   
3  I was stuck between my ex-wife and my family, ...         family   
4  I had a hard time deciding where to be assigne...         career   

   count_cat_career  count_cat_relationships  count_cat_family  \
0                 0                       10                 3   
1                 0                        0                 0   
2                 1                        0                 1   
3                 0                        2                 3   
4                 0                        0                 0   

   count_cat_health  count_cat_financial  count_cat_education  \
0                 0                    0                    1   
1                 1                    0      

In [13]:
from transformers import pipeline
import pandas as pd
import time

# Define categories
DEC_CATS = {
    'career': [], 'relationships': [], 'family': [], 'health': [],
    'financial': [], 'education': [], 'location': [], 'ethical': []
}
candidate_labels = list(DEC_CATS.keys())

# Load zero-shot classifier once
zsc = pipeline(
    "zero-shot-classification",
    model="facebook/bart-large-mnli",
    device=-1  # Use CPU
)

# Function to classify by country with progress tracking
def run_llm_classification_by_country(country_name, save=True, batch_size=50):
    df_country = text_combined_df[text_combined_df["country"] == country_name].copy()
    texts = df_country["Open_Q1_Q7"].tolist()

    all_results = []
    total = len(texts)

    for i in range(0, total, batch_size):
        batch = texts[i:i + batch_size]
        try:
            res = zsc(batch, candidate_labels=candidate_labels, multi_label=True)
            all_results.extend(res)
            print(f"[{time.ctime()}] Processed {min(i + batch_size, total)} / {total} for {country_name}")
        except Exception as e:
            print(f"⚠️ Batch {i}-{i + batch_size} failed: {e}")
            continue

    # Extract predictions
    df_country["llm_category"] = [r["labels"][0] for r in all_results]

    score_df = pd.DataFrame([
        {f"llm_cat_{label}": score for label, score in zip(r["labels"], r["scores"])}
        for r in all_results
    ])

    df_country = pd.concat([df_country.reset_index(drop=True), score_df], axis=1)

    if save:
        filename = f"llm_zero_shot_{country_name}.xlsx"
        df_country.to_excel(filename, index=False)
        print(f"✔ Finished and saved: {filename}")

    return df_country


Device set to use cpu


In [22]:
turkey_df = run_llm_classification_by_country("USA")

[Wed Jun 18 10:15:22 2025] Processed 50 / 151 for USA
[Wed Jun 18 10:17:24 2025] Processed 100 / 151 for USA
[Wed Jun 18 10:19:28 2025] Processed 150 / 151 for USA
[Wed Jun 18 10:19:29 2025] Processed 151 / 151 for USA
✔ Finished and saved: llm_zero_shot_USA.xlsx
